# Examples

## 低阈值与高阈值钙通道的门控特性对比

本实例将展示如何使用 `braincell` 库分析低阈值 T 型钙通道和高阈值钙通道的门控变量稳态特性。通过对比它们的门控曲线，你将理解不同钙通道在神经元电活动中的功能差异。

### 准备工作

首先确保已安装必要的库（`braincell`、`brainunit`、`braintools`、`matplotlib`），并导入所需模块：

```python
import braintools
import brainunit as u  # 神经科学常用单位
import matplotlib.pyplot as plt  # 基础绘图库
import braincell  # 核心库:用于构建和模拟离子通道
```

### 代码详解

#### 创建钙通道模型

`braincell` 内置了多种基于经典文献的离子通道模型，命名格式为 `通道类型_文献标识`，如 `ICaT_HP1992` 表示基于 Huguenard & Prince 1992 年研究的 T 型钙通道。
如果你想了解更多的离子通道模型，可以查阅我们的[离子通道模型库](https://braincell.readthedocs.io/latest/apis/braincell.channel.html)。

```python
# 创建低阈值 T 型钙通道（ICaT）
cat = braincell.channel.ICaT_HP1992(1)

# 创建高阈值钙通道（ICaHT）
caht = braincell.channel.ICaHT_HM1992(1)
```

#### 生成电压序列

为了分析通道门控特性随膜电位的变化，我们生成一个从 -100 mV 到 0 mV 的连续电压序列：

```python
# 生成电压序列
vs = u.math.arange(-100 * u.mV, 0 * u.mV, 0.1 * u.mV)
```

#### 计算门控变量稳态值并绘图

离子通道的门控变量的稳态值描述了在某一膜电位下，通道门控处于开放状态的概率。我们通过通道的 `f_q_inf` 和 `f_p_inf` 方法计算这些值，并可视化对比：

```python
# 创建图形和子图布局
fig, gs = braintools.visualize.get_figure(1, 2, 3., 4.5)

# 计算低阈值通道的激活门稳态值和失活门稳态值
q_inf = cat.f_q_inf(vs)
p_inf = cat.f_p_inf(vs)

# 添加子图 1
fig.add_subplot(gs[0, 0])
plt.plot(vs / u.mV, q_inf, label='q_inf（激活门）')  # 横轴转换为 mV 便于阅读
plt.plot(vs / u.mV, p_inf, label='p_inf（失活门）')
plt.legend()  # 显示图例
plt.fill_between([-80, -60], 1., alpha=0.2)  # 标记低阈值通道的典型激活范围
plt.title('低阈值钙通道（ICaT）')
plt.xlabel('膜电位（mV）')

# 计算高阈值通道的 q_inf 和 p_inf
q_inf = caht.f_q_inf(vs)
p_inf = caht.f_p_inf(vs)

# 添加子图 2
fig.add_subplot(gs[0, 1])
plt.plot(vs / u.mV, q_inf, label='q_inf（激活门）')
plt.plot(vs / u.mV, p_inf, label='p_inf（失活门）')
plt.fill_between([-60, -40], 1., alpha=0.2)  # 标记高阈值通道的典型激活范围
plt.legend()
plt.xlabel('膜电位（mV）')
plt.title('高阈值钙通道（ICaHT）')

# 显示图形
plt.show()
```

### 结果解读

运行代码后，你将看到两张门控特性曲线图，如下：

![门控特性曲线图](../_static/calciumactivationfunctions.png)

分析两张门控特性曲线图，核心结论如下：

低阈值钙通道（：
   - `q_inf`在膜电位约 -80 ~ -60 mV 时显著上升，表明此范围内通道易激活。
   - 与填充区域吻合，说明该通道在神经元静息电位附近即可被激活。

高阈值钙通道）：
   - `q_inf` 在膜电位约 -60 ~ -40 mV 时才显著上升，激活阈值更高。
   - 填充区域对应其典型激活范围，需更强去极化才能激活。

### 扩展练习

- 尝试修改通道的电导密度参数，观察门控曲线是否变化。
- 对比其他离子通道的门控特性，理解不同通道在动作电位中的协同作用。

通过这些工具，你可以快速构建符合电生理特性的通道模型，为复杂神经动力学模拟奠定基础。

## 兴奋性-抑制性神经元网络（E-I网络）的模拟与分析

本实例将基于 `braincell` 框架实现一个经典的兴奋-抑制神经元网络。通过构建由 Hodgkin-Huxley模型神经元组成的 E-I 网络，你将学习如何在 `braincell` 中实现从单神经元到网络的层级建模，并分析网络的 spike 动态特性。


### 准备工作
首先确保已安装必要的库（`braincell`、`brainstate`、`brainunit`、`matplotlib`），并导入所需模块：

```python
import brainunit as u  # 神经科学常用单位
import matplotlib.pyplot as plt  # 基础绘图库
import brainstate  # 用于网络动态管理和仿真
import braincell  # 核心库：构建神经元和离子通道
```

### 代码详解

#### 参数定义

首先定义关键物理参数，这些参数决定了神经元的电生理特性和网络规模：

```python
# 神经元动作电位发放阈值
V_th = -20. * u.mV

# 神经元膜面积
area = 20000 * u.um **2
area = area.in_unit(u.cm** 2)

# 膜电容
Cm = (1 * u.uF * u.cm ** -2) * area  # 总电容 = 比电容 × 面积
```

#### 定义 HH 单神经元模型

使用 `SingleCompartment` 构建基于 HH 模型的单神经元，包含钠通道 `INa` 、钾通道 `IK` 和漏电流 `IL` ，这些通道共同决定神经元的放电特性：

```python
class HH(braincell.SingleCompartment):
    def __init__(self, in_size):
        # 初始化单房室神经元
        super().__init__(in_size, C=Cm, solver='ind_exp_euler')

        # 钠离子通道（INa）
        self.na = braincell.ion.SodiumFixed(in_size, E=50. * u.mV)
        self.na.add_elem(
            # 最大电导
            INa=braincell.channel.INa_TM1991(in_size, g_max=(100. * u.mS * u.cm **-2) * area, V_sh=-63. * u.mV)
        )

        # 钾离子通道（IK）
        self.k = braincell.ion.PotassiumFixed(in_size, E=-90 * u.mV)
        self.k.add_elem(
            # 最大电导
            IK=braincell.channel.IK_TM1991(in_size, g_max=(30. * u.mS * u.cm** -2) * area, V_sh=-63. * u.mV)
        )

        # 漏电流（IL）
        self.IL = braincell.channel.IL(
            in_size,
            E=-60. * u.mV,
            g_max=(5. * u.nS * u.cm **-2) * area  # 最大电导
        )
```

#### 定义 E-I 网络：兴奋性与抑制性神经元的连接

构建由兴奋性（E）和抑制性（I）神经元组成的网络，模拟皮层网络中常见的 E-I 平衡机制：

```python
class EINet(brainstate.nn.DynamicsGroup):
    def __init__(self):
        super().__init__()
        # 网络规模
        self.n_exc = 3200
        self.n_inh = 800
        self.num = self.n_exc + self.n_inh  # 总神经元数：4000

        # 初始化神经元群体
        self.N = HH(self.num)

        # 兴奋性突触投射
        self.E = brainstate.nn.AlignPostProj(
            # 连接规则
            comm=brainstate.nn.EventFixedProb(
                self.n_exc, self.num, conn_num=0.02,  # 连接概率
                conn_weight=6. * u.nS  # 突触权重
            ),
            # 突触动力学
            syn=brainstate.nn.Expon(self.num, tau=5. * u.ms),
            # 突触后效应
            out=brainstate.nn.COBA(E=0. * u.mV),
            post=self.N  # 投射目标为神经元群体 N
        )

        # 抑制性突触投射
        self.I = brainstate.nn.AlignPostProj(
            # 连接规则
            comm=brainstate.nn.EventFixedProb(
                self.n_inh, self.num, conn_num=0.02,
                conn_weight=67. * u.nS
            ),
            # 突触动力学
            syn=brainstate.nn.Expon(self.num, tau=10. * u.ms),
            # 突触后效应
            out=brainstate.nn.COBA(E=-80. * u.mV),
            post=self.N  # 投射目标为神经元群体 N
        )

    def update(self, t):
        # 定义网络随时间的更新规则
        with brainstate.environ.context(t=t):
            # 获取当前时刻的 spike 信号
            spk = self.N.spike.value

            # 兴奋性神经元的 spike 驱动兴奋性突触投射
            self.E(spk[:self.n_exc])

            # 抑制性神经元的 spike 驱动抑制性突触投射
            self.I(spk[self.n_exc:])

            # 神经元接收突触输入后更新状态，返回新的 spike 信号
            spk = self.N(0. * u.nA)
            return spk
```

#### 运行网络模拟

初始化网络并运行仿真，记录每个时刻的神经元 spike 活动：

```python
# 初始化 E-I 网络
net = EINet()
brainstate.nn.init_all_states(net)  # 初始化网络中所有神经元和突触的状态

# 设置仿真参数并运行
with brainstate.environ.context(dt=0.1 * u.ms):  # 时间步长
    # 生成仿真时间序列
    times = u.math.arange(0. * u.ms, 100. * u.ms, brainstate.environ.get_dt())

    # 循环更新网络状态
    spikes = brainstate.compile.for_loop(
        net.update, times,
        pbar=brainstate.compile.ProgressBar(10)  # 显示进度条
    )
```

#### 可视化网络 spike 活动

将 spike 数据绘制成格栅图，直观展示神经元在时间上的发放模式：

```python
# 提取 spike 发生的时间和神经元索引
t_indices, n_indices = u.math.where(spikes)

# 绘制 raster plot
plt.scatter(times[t_indices], n_indices, s=1)
plt.xlabel('Time (ms)')  # 横轴：时间
plt.ylabel('Neuron index')  # 纵轴：神经元索引
plt.show()
```

### 结果解读

运行代码后，你将看到一张格栅图，如下图：

![raster图](../_static/cobahh2007.png)

其中每个点代表一个神经元在特定时间的 spike 发放。典型的 E-I 网络动态具有以下特征：
- 异步发放：神经元发放时间分散，无明显同步节律。
- 稀疏活动：大多数神经元在 100 ms 内发放次数较少。
- 无爆发式同步：由于抑制性突触的快速反馈，避免了大规模神经元同步爆发。

这些特征表明 E-I 网络通过兴奋性和抑制性的动态平衡，维持了稳定且符合生理特性的活动模式。

### 扩展练习

- 调整抑制性突触权重，观察网络是否出现过度同步，即爆发式活动。
- 增加兴奋性神经元比例，分析 E-I 失衡对网络动态的影响。
- 延长仿真时间，观察网络是否维持稳定的异步活动。

通过这些扩展，你可以深入理解 E-I 平衡在维持神经网络功能中的核心作用，以及 ``braincell`` 在复杂网络建模中的灵活性。

## 不同积分方法对 HH 模型神经元动态的影响

本实例将通过实现经典的 Hodgkin-Huxley 模型，对比指数欧拉法 ``exp_euler`` 和独立指数欧拉法 ``ind_exp_euler`` 对神经元膜电位动态的影响。
你将学习如何在 `braincell` 中配置不同的积分器，以及它们在模拟神经元电活动时的差异。

### 准备工作

首先确保已安装必要的库（`braincell`、`brainstate`、`brainunit`、`matplotlib`），并导入所需模块：

```python
import brainstate
import brainunit as u
import matplotlib.pyplot as plt

import braincell
```

### 代码详解

#### 定义 HH 模型神经元

使用 `SingleCompartment` 构建基于 HH 模型的单神经元，包含钠通道 `INa` 、钾通道 `IK` 和漏电流 `IL` ，这些通道共同决定神经元的放电特性：

```python
class HH(braincell.SingleCompartment):
    def __init__(self, size, solver='rk4'):
        super().__init__(size, solver=solver)

        self.na = braincell.ion.SodiumFixed(size, E=50. * u.mV)
        self.na.add(INa=braincell.channel.INa_HH1952(size))

        self.k = braincell.ion.PotassiumFixed(size, E=-77. * u.mV)
        self.k.add(IK=braincell.channel.IK_HH1952(size))

        self.IL = braincell.channel.IL(
            size,
            E=-54.387 * u.mV,
            g_max=0.03 * (u.mS / u.cm **2)
        )
```

#### 初始化神经元与配置积分方法

创建两个 HH 神经元实例，分别使用标准指数欧拉法 ``exp_euler``和独立指数欧拉法 ``ind_exp_euler``，并初始化神经元状态：

```python
# 创建使用标准指数欧拉法的 HH 神经元
hh_exp = HH(1, solver='exp_euler')

# 创建使用独立指数欧拉法的 HH 神经元
hh_ind_exp = HH(1, solver='ind_exp_euler')

# 初始化神经元状态（如膜电位、门控变量等）至静息状态
hh_exp.init_state()
hh_ind_exp.init_state()
```

#### 定义模拟步骤函数

编写函数描述神经元在每个时间步的更新规则，包括接收输入电流并返回膜电位：

```python
def step_fun(t, neuron):
    # 在当前时间 t 下更新神经元状态
    with brainstate.environ.context(t=t):
        # 向神经元注入持续电流以触发动作电位
        neuron.update(10 * u.nA / u.cm** 2)
    # 返回当前膜电位值
    return neuron.V.value
```

#### 运行模拟并记录结果

配置模拟参数，分别运行两种积分方法的模拟，记录膜电位随时间的变化：

```python
# 设置模拟时间步长
with brainstate.environ.context(dt=0.1 * u.ms):
    # 生成模拟时间序列
    times = u.math.arange(0. * u.ms, 100 * u.ms, brainstate.environ.get_dt())

    # 用标准指数欧拉法模拟，记录膜电位
    vs_exp = brainstate.compile.for_loop(
        lambda t: step_fun(t, hh_exp),
        times
    )

    # 用独立指数欧拉法模拟，记录膜电位
    vs_ind_exp = brainstate.compile.for_loop(
        lambda t: step_fun(t, hh_ind_exp),
        times
    )
```

#### 可视化膜电位动态

绘制两种积分方法下的膜电位变化曲线，对比动作电位波形差异：

```python
# 绘制标准指数欧拉法的结果
plt.plot(times, u.math.squeeze(vs_exp), label='exp_euler', linewidth=1.5)

# 绘制独立指数欧拉法的结果
plt.plot(times, u.math.squeeze(vs_ind_exp), label='ind_exp_euler', linestyle='--', linewidth=1.5)

# 添加标签和图例
plt.xlabel('Time (ms)')
plt.ylabel('Membrane Potential (mV)')
plt.legend()
plt.title('HH Model Dynamics with Different Integrators')
plt.show()
```

### 结果解读

运行代码后，你将看到膜电位随时间变化的曲线：

![膜电位随时间变化曲线](../_static/hhneuron.png)

核心结论如下：
动作电位基本特征：两种方法均能模拟出动作电位的产生，但波形细节存在差异。
积分方法差异：
   - `exp_euler` 曲线更平滑，对强耦合的门控变量与膜电位关系拟合更精确，动作电位峰值和时程更接近理论值。
   - `ind_exp_euler` 由于独立更新状态变量，可能在快速变化阶段出现微小偏差，但计算速度更快，尤其在大规模模拟时。
适用性场景：
   - 对精度要求高的单细胞模拟，如复现特定电生理实验，优先选择 `exp_euler` 。
   - 大规模网络模拟，如包含数千神经元，`ind_exp_euler` 能在保证足够精度的同时显著提升效率。

### 扩展练习

- 尝试使用四阶龙格-库塔法 `solver='rk4'` ，对比其与指数欧拉法的精度和计算效率。
- 调整输入电流强度，观察不同积分方法对神经元发放频率的影响。
- 增加模拟时长，分析长期模拟中积分误差的累积效应。

通过这些实践，你将深入理解数值方法在神经模拟中的关键作用，为复杂模型设计提供依据。

### 更多

若想了解更多的数值积分方法，我们在[数值积分方法库](https://braincell.readthedocs.io/latest/apis/integration.html) 中提供了丰富的积分器与相关资料。

## 丘脑神经元模型的构建与电活动模拟

本实例将基于 `braincell` 框架实现丘脑神经元模型，包括四种典型丘脑神经元类型： `HTC`、`RTC`、`IN`、`TRN`。
通过构建这些包含复杂离子通道组合的神经元模型，你将学习如何模拟丘脑神经元的电生理特性，理解不同神经元类型的通道组成差异及其功能意义。

### 准备工作
首先确保已安装必要的库（`braincell`、`brainstate`、`brainunit`、`matplotlib`），并导入所需模块：

```python
import time  # 用于记录模拟耗时

import brainstate
import braintools
import brainunit as u
import matplotlib.pyplot as plt
import braincell
```

### 代码详解

#### 丘脑神经元基类

定义所有丘脑神经元的基类，封装通用功能：

```python
class ThalamusNeuron(braincell.SingleCompartment):
    def compute_derivative(self, x=0. * u.nA):
        # 导数计算方法
        return super().compute_derivative(x * self.area)

    def step_run(self, t, inp):
        # 定义每个时间的更新规则
        with brainstate.environ.context(t=t):
            self.update(inp)
            return self.V.value
```

#### 四种丘脑神经元类型的实现

在丘脑中，不同神经元类型通过特定的离子通道组合实现不同功能。以下分别解析其通道组成：

##### HTC

```python
class HTC(ThalamusNeuron):
    def __init__(
        self,
        size,
        gKL=0.01 * (u.mS / u.cm **2),  # 钾漏通道电导
        V_initializer=brainstate.init.Constant(-65. * u.mV),  # 初始膜电位
        solver: str = 'ind_exp_euler'  # 积分方法
    ):
        super().__init__(size, V_initializer=V_initializer, V_th=20. * u.mV, solver=solver)

        # 膜面积参数
        self.area = 1e-3 / (2.9e-4 * u.cm** 2)

        # 钠离子通道
        self.na = braincell.ion.SodiumFixed(size, E=50. * u.mV)  # 钠反转电位 50 mV
        self.na.add(INa=braincell.channel.INa_Ba2002(size, V_sh=-30 * u.mV))

        # 钾离子通道
        self.k = braincell.ion.PotassiumFixed(size, E=-90. * u.mV)  # 钾反转电位 -90 mV
        self.k.add(IKL=braincell.channel.IK_Leak(size, g_max=gKL))  # 钾漏电流
        self.k.add(IDR=braincell.channel.IKDR_Ba2002(size, V_sh=-30. * u.mV, phi=0.25))  # 延迟整流钾电流

        # 钙离子通道
        self.ca = braincell.ion.CalciumDetailed(
            size,
            C_rest=5e-5 * u.mM,  # 静息钙浓度
            tau=10. * u.ms,  # 钙浓度衰减时间常数
            d=0.5 * u.um  # 钙扩散距离
        )
        self.ca.add(ICaL=braincell.channel.ICaL_IS2008(size, g_max=0.5 * (u.mS / u.cm **2)))  # L型钙通道
        self.ca.add(ICaN=braincell.channel.ICaN_IS2008(size, g_max=0.5 * (u.mS / u.cm** 2)))  # N型钙通道
        self.ca.add(ICaT=braincell.channel.ICaT_HM1992(size, g_max=2.1 * (u.mS / u.cm **2)))  # T型钙通道（低阈值）
        self.ca.add(ICaHT=braincell.channel.ICaHT_HM1992(size, g_max=3.0 * (u.mS / u.cm** 2)))  # 高阈值钙通道

        # 钙激活钾通道（IAHP）
        self.kca = braincell.MixIons(self.k, self.ca)  # 混合钾离子和钙离子
        self.kca.add(IAHP=braincell.channel.IAHP_De1994(size, g_max=0.3 * (u.mS / u.cm **2)))

        # 超极化激活阳离子电流（Ih）和漏电流（IL）
        self.Ih = braincell.channel.Ih_HM1992(size, g_max=0.01 * (u.mS / u.cm** 2), E=-43 * u.mV)  # 调节静息电位和节律
        self.IL = braincell.channel.IL(size, g_max=0.0075 * (u.mS / u.cm **2), E=-70 * u.mV)  # 背景漏电流
```

##### RTC

```python
class RTC(ThalamusNeuron):
    def __init__(
        self,
        size,
        gKL=0.01 * (u.mS / u.cm** 2),
        V_initializer=brainstate.init.Constant(-65. * u.mV),
        solver: str = 'ind_exp_euler'
    ):
        super().__init__(size, V_initializer=V_initializer, V_th=20 * u.mV, solver=solver)

        self.area = 1e-3 / (2.9e-4 * u.cm **2)  # 膜面积参数

        # 钠离子通道
        self.na = braincell.ion.SodiumFixed(size)
        self.na.add(INa=braincell.channel.INa_Ba2002(size, V_sh=-40 * u.mV))

        # 钾离子通道
        self.k = braincell.ion.PotassiumFixed(size, E=-90. * u.mV)
        self.k.add(IDR=braincell.channel.IKDR_Ba2002(size, V_sh=-40 * u.mV, phi=0.25))  # 门控偏移
        self.k.add(IKL=braincell.channel.IK_Leak(size, g_max=gKL))

        # 钙离子通道
        self.ca = braincell.ion.CalciumDetailed(size, C_rest=5e-5 * u.mM, tau=10. * u.ms, d=0.5 * u.um)
        self.ca.add(ICaL=braincell.channel.ICaL_IS2008(size, g_max=0.3 * (u.mS / u.cm** 2)))
        self.ca.add(ICaN=braincell.channel.ICaN_IS2008(size, g_max=0.6 * (u.mS / u.cm **2)))
        self.ca.add(ICaT=braincell.channel.ICaT_HM1992(size, g_max=2.1 * (u.mS / u.cm** 2)))
        self.ca.add(ICaHT=braincell.channel.ICaHT_HM1992(size, g_max=0.6 * (u.mS / u.cm **2)))

        # 钙激活钾通道（IAHP）
        self.kca = braincell.MixIons(self.k, self.ca)
        self.kca.add(IAHP=braincell.channel.IAHP_De1994(size, g_max=0.1 * (u.mS / u.cm** 2)))

        # Ih 和 IL 电流
        self.Ih = braincell.channel.Ih_HM1992(size, g_max=0.01 * (u.mS / u.cm **2), E=-43 * u.mV)
        self.IL = braincell.channel.IL(size, g_max=0.0075 * (u.mS / u.cm** 2), E=-70 * u.mV)
```

##### IN

```python
class IN(ThalamusNeuron):
    def __init__(
        self,
        size,
        gKL=0.01 * (u.mS / u.cm **2),
        V_initializer=brainstate.init.Constant(-70. * u.mV),  # 初始膜电位更负
        solver: str = 'ind_exp_euler'
    ):
        super().__init__(size, V_initializer=V_initializer, V_th=20. * u.mV, solver=solver)

        self.area = 1e-3 / (1.7e-4 * u.cm** 2)

        # 钠离子通道
        self.na = braincell.ion.SodiumFixed(size)
        self.na.add(INa=braincell.channel.INa_Ba2002(size, V_sh=-30 * u.mV))

        # 钾离子通道
        self.k = braincell.ion.PotassiumFixed(size, E=-90. * u.mV)
        self.k.add(IDR=braincell.channel.IKDR_Ba2002(size, V_sh=-30 * u.mV, phi=0.25))
        self.k.add(IKL=braincell.channel.IK_Leak(size, g_max=gKL))

        # 钙离子通道
        self.ca = braincell.ion.CalciumDetailed(size, C_rest=5e-5 * u.mM, tau=10. * u.ms, d=0.5 * u.um)
        self.ca.add(ICaN=braincell.channel.ICaN_IS2008(size, g_max=0.1 * (u.mS / u.cm **2)))
        self.ca.add(ICaHT=braincell.channel.ICaHT_HM1992(size, g_max=2.5 * (u.mS / u.cm** 2)))

        # 钙激活钾通道（IAHP）
        self.kca = braincell.MixIons(self.k, self.ca)
        self.kca.add(IAHP=braincell.channel.IAHP_De1994(size, g_max=0.2 * (u.mS / u.cm **2)))

        # 漏电流（IL）和 Ih 电流
        self.IL = braincell.channel.IL(size, g_max=0.0075 * (u.mS / u.cm** 2), E=-60 * u.mV)
        self.Ih = braincell.channel.Ih_HM1992(size, g_max=0.05 * (u.mS / u.cm **2), E=-43 * u.mV)
```

##### TRN

```python
class TRN(ThalamusNeuron):
    def __init__(
        self,
        size,
        gKL=0.01 * (u.mS / u.cm** 2),
        V_initializer=brainstate.init.Constant(-70. * u.mV),
        gl=0.0075,  # 漏通道电导系数
        solver: str = 'ind_exp_euler'
    ):
        super().__init__(size, V_initializer=V_initializer, V_th=20. * u.mV, solver=solver)

        self.area = 1e-3 / (1.43e-4 * u.cm **2)

        # 钠离子通道
        self.na = braincell.ion.SodiumFixed(size)
        self.na.add(INa=braincell.channel.INa_Ba2002(size, V_sh=-40 * u.mV))

        # 钾离子通道
        self.k = braincell.ion.PotassiumFixed(size, E=-90. * u.mV)
        self.k.add(IDR=braincell.channel.IKDR_Ba2002(size, V_sh=-40 * u.mV))
        self.k.add(IKL=braincell.channel.IK_Leak(size, g_max=gKL))

        # 钙离子通道
        self.ca = braincell.ion.CalciumDetailed(size, C_rest=5e-5 * u.mM, tau=100. * u.ms, d=0.5 * u.um)
        self.ca.add(ICaN=braincell.channel.ICaN_IS2008(size, g_max=0.2 * (u.mS / u.cm** 2)))
        self.ca.add(ICaT=braincell.channel.ICaT_HP1992(size, g_max=1.3 * (u.mS / u.cm **2)))

        # 钙激活钾通道（IAHP）
        self.kca = braincell.MixIons(self.k, self.ca)
        self.kca.add(IAHP=braincell.channel.IAHP_De1994(size, g_max=0.2 * (u.mS / u.cm** 2)))

        # 漏电流（IL）
        self.IL = braincell.channel.IL(size, g_max=gl * (u.mS / u.cm **2), E=-60 * u.mV)
```

#### 模拟丘脑神经元电活动

定义 `try_trn_neuron` 函数，设置输入电流、模拟参数，运行仿真并可视化结果：

```python
def try_trn_neuron():
    # 设置模拟时间步长
    brainstate.environ.set(dt=0.02 * u.ms)

    # 生成分段输入电流
    I = braintools.input.section_input(
        values=[0, 0.05, 0],  # 电流强度
        durations=[50 * u.ms, 200 * u.ms, 100 * u.ms]  # 各阶段持续时间
    ) * u.uA

    # 生成模拟时间序列
    times = u.math.arange(I.shape[0]) * brainstate.environ.get_dt()

    # 选择模拟的神经元类型（此处以 HTC 为例，可替换为 RTC、IN、TRN）
    neu = HTC(1, solver='ind_exp_euler')  # 使用独立指数欧拉法积分
    neu.init_state()  # 初始化神经元状态

    # 运行模拟并记录耗时
    t0 = time.time()
    vs = brainstate.compile.for_loop(neu.step_run, times, I)  # 循环更新神经元状态
    t1 = time.time()
    print(f"模拟耗时：{t1 - t0:.4f} 秒")  # 输出计算效率

    # 可视化膜电位变化
    plt.plot(
        times.to_decimal(u.ms),  # 横轴：时间
        u.math.squeeze(vs.to_decimal(u.mV))  # 纵轴：膜电位
    )
    plt.xlabel('时间（ms）')
    plt.ylabel('膜电位（mV）')
    plt.title('HTC 神经元对分段电流的响应')
    plt.show()


# 执行模拟
if __name__ == '__main__':
    try_trn_neuron()
```

### 结果解读

运行代码后，将得到 HTC 神经元在分段电流刺激下的膜电位变化曲线，典型特征如下：

![膜电位变化曲线](../_static/thalamussinglecompartment.png)

0~50 ms（无电流）：神经元处于静息状态，膜电位稳定在约 -65 mV，由漏电流和 Ih 电流平衡维持。

50~250 ms（注入 0.05 uA 电流）：
   - 电流注入导致膜电位去极化，达到阈值后产生动作电位。
   - 由于 T 型钙通道和 IAHP 电流的作用，可能出现爆发式放电，这是丘脑神经元参与节律振荡的典型特征。

250~350 ms（电流关闭）：
   - 膜电位逐渐恢复静息水平，但可能因钙激活钾通道的持续作用出现超极化后电位。

### 扩展练习

- 替换神经元类型，如 ``neu = TRN(1)``，对比 TRN 与 HTC 对相同电流的响应差异。
- 修改 T 型钙通道电导，如 ``ICaT_HM1992(g_max=1.0 * ...)``，观察对神经元爆发式放电的影响。
- 延长模拟时间并分析不同丘脑神经元的自发节律特性。

通过这些实践，你将深入理解丘脑神经元的电生理特性及其在大脑节律调控中的作用。